In [ ]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   


In [2]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, state, _ = fbg.eval_global_conn_k(states, G_base)
print("k =", k_val, "state =", state)

k = 2 state = 2


In [3]:
s_fun = lambda comps_st: fbg.eval_global_conn_k(comps_st, G_base)
row_names = list(edges.keys()) 
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [4]:
sys_surv_st = 1 # either 1 or 2

result = tsum.run_rule_extraction_by_mcs(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    sys_surv_st=sys_surv_st,
    unk_prob_thres = 1e-6,
    output_dir="tsum_res"

) 

---
Round: 1, Unk. prob.: 1.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 1, System value: 1. Total samples: 100000.
New rule (No. of conditions: 7): {'e01': ('>=', 1), 'e02': ('>=', 1), 'e05': ('>=', 1), 'e06': ('>=', 1), 'e07': ('>=', 1), 'e08': ('>=', 1), 'e09': ('>=', 1), 'sys': ('>=', 1)}
Updated sys_vals: [1]
---
Round: 2, Unk. prob.: 1.000e+00
Surv probs: 0.000e+00, Fail probs: 0.000e+00
No. of non-dominant rules: 1, Survival rules: 1, Failure rules: 0
Survival sample found from sampling.
No. of existing rules removed:  0
New rule added. System state: 1, System value: 1. Total samples: 100000.
New rule (No. of conditions: 7): {'e03': ('>=', 1), 'e04': ('>=', 1), 'e05': ('>=', 1), 'e06': ('>=', 1), 'e07': ('>=', 1), 'e09': ('>=', 1), 'e11': ('>=', 1), 'sys': ('>=', 1)}
---
Round: 3, Unk. prob.: 7.896e-01
Surv probs: 2.104e-01, Fail probs: 0.000e+00
No. 